In [1]:
from transformers import pipeline
import os
import pandas as pd
import datasets
import gc
from tqdm import tqdm

In [2]:
model_ckpt2= 'circulus/kobart-trans-ko-en-v2' #bart`
pipe = pipeline('translation', model = model_ckpt2, device=0,max_length=256)

In [7]:
# test
pipe('인간 바둑기사가 인공지능에 승리할 수 있었던 이유는 인공지능으로부터 전술을 직접 물려 받았기 때문이다.')

[{'translation_text': 'The reason why human Go players were able to win artificial intelligence was because they inherited tactics directly from artificial intelligence'}]

In [64]:
# load data
data_path = './data/'
df = pd.read_csv(data_path) # 데이터 파일 불러오기
df.drop_duplicates(inplace=True)

In [151]:
# translate data and save
def data_save(data, pipe, prefix, num_rows,start, directory) :
    
    '''
    
    1. `dataset` : 데이터 셋 [dataframe : pandas dataframe]
    2. `pipe` : hugging face pipeline [func : huggingface pipeline function]
    3. `지역` : 사투리 지역 [str : 'jj', 'gs', 'kw', 'cc', 'jd']
    4. `작업크기` : num of rows [int]
    5. `directory` : 저장 경로 [str : directory]
    
        예) data_save(df,pipe,'jj',5000,0,'/aiffel/aiffelthon/') -> '지역', 시작index만 변경해주시면 됩니다.
        
    '''
    
    df_final = pd.DataFrame(columns=['text', 'dial', 'reg', 'pair','eng'])

    for i in range(start,len(data), num_rows) :
        dataset = data[i : i +num_rows].reset_index()
        dataset = datasets.Dataset.from_pandas(dataset)
        dataset = dataset.map(lambda ds : {'eng' : pipe(ds['text'])[0]['translation_text']})
        dataset.set_format('pandas')
        df_final = df_final.merge(dataset[:], how='outer')
        df_final.to_csv(directory + f'dataset_en_ko_{prefix}_{i}_{i+num_rows}.csv', index=False)
        print(f'작업 완료 : {prefix}_{i}_{i+num_rows}' )
        print(f'저장 경로 : {directory}dataset_ko_en_{prefix}_{i}_{i+num_rows}.csv')